## Load_Tokenization

In [6]:
import pandas as pd
import numpy as np
import re
import pickle

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

In [4]:
df = pd.read_csv('./data_l/ssodam2021_tokenized.csv',encoding='utf-8')

In [18]:
df.iloc[0]

num                                                                    1082901
board                                                                      익게2
title                                                                    해피뉴이어
text                                                      2021년은 모두 행복한 한해가 되길
writer                                                                      익명
upload_date                                                         2021/01/01
upload_time                                                              00:00
view                                                                    1343.0
likes                                                                    195.0
dislikes                                                                   0.0
comments                     앗, 성지다, 첫글 ㅊㅊ, 추천 준다, 내 24살은 인생에서 가장 빛나는 날로 만들...
comments_writer                                                            NaN
comments_cnt                                        

### Preprocessing

In [19]:
def get_noun_list_from_str(msg):
    try :
        return [tuple([re.sub("'",'',y) for y in re.findall('\'.*?\'',x)]) for x in re.findall('\(.*?\)',msg)]
    except :
        return np.NaN

In [20]:
df = df.fillna('')
df.text_tokenized = df.text_tokenized.apply(lambda x: get_noun_list_from_str(x))
df.title_tokenized = df.title_tokenized.apply(lambda x: get_noun_list_from_str(x))
df.comments_tokenized = df.comments_tokenized.apply(lambda x: get_noun_list_from_str(x))
df.comments_writer_tokenized = df.comments_writer_tokenized.apply(lambda x: get_noun_list_from_str(x))
df.upload_date = pd.to_datetime(df.upload_date)

In [82]:
df.head()

,num,board,title,text,writer,upload_date,upload_time,view,likes,dislikes,comments,comments_writer,comments_cnt,text_tokenized,title_tokenized,comments_tokenized,comments_writer_tokenized
0,1082901,익게2,해피뉴이어,2021년은 모두 행복한 한해가 되길,익명,2021-01-01,00:00,1343.0,195.0,0.0,"앗, 성지다, 첫글 ㅊㅊ, 추천 준다, 내 24살은 인생에서 가장 빛나는 날로 만들...",,20.0,"[(2021년, Number), (은, Foreign), (모두, Noun), (행...","[(해피뉴이어, Noun)]","[(앗, Noun), (,, Punctuation), (성지, Noun), (다, ...",[]
1,1082902,익게2,2021년 새해복 많이받으세요,모든 일이 잘 되기를 12시 땡,익명,2021-01-01,00:00,107.0,3.0,0.0,늦었네,"2빠다 ㅎㅎ, 내년에 도전한다",3.0,"[(모든, Noun), (일이, Modifier), (잘, Verb), (되, Ve...","[(2021년, Number), (새해, Noun), (복, Noun), (많이, ...","[(늦었네, Verb)]","[(2, Number), (빠다, Noun), (ㅎㅎ, KoreanParticle)..."
2,1082903,익게2,첫글은 내꼬,예비회계사 나다미,익명,2021-01-01,00:00,139.0,1.0,0.0,"ㄹㅇ 1등이네 ㅋㅋ, 실패",ㅜㅜ,3.0,"[(예비, Noun), (회계사, Noun), (나, Noun), (다미, Noun)]","[(첫, Modifier), (글, Noun), (은, Josa), (내꼬, Noun)]","[(ㄹㅇ, KoreanParticle), (1등, Number), (이네, Fore...","[(ㅜㅜ, KoreanParticle)]"
3,1082905,익게2,땡ㅎㅎㅎㅎㅎㅎㅎ,1등,익명,2021-01-01,00:00,39.0,0.0,0.0,,,0.0,"[(1등, Number)]","[(땡, Noun), (ㅎㅎㅎㅎㅎㅎㅎ, KoreanParticle)]",[],[]
4,1082906,익게2,어디 카운트 다운 하는 곳 없냐,언제바껴,익명,2021-01-01,00:00,84.0,0.0,0.0,232번 불교방송,,1.0,"[(언, Modifier), (제바, Noun), (껴, Verb)]","[(어디, Noun), (카운트, Noun), (다운, Noun), (하는, Ver...","[(232, Number), (번, Noun), (불교, Noun), (방송, No...",[]


In [7]:
with open('./data_l/ssodam2021_tokenized_df.pkl','rb') as f:
    df = pickle.load(f)

## BoW

In [9]:
#불용어 load
with open('./data/stopwords.txt',encoding='utf-8') as f:
    stopwords = f.read().split()

In [370]:
#형태소 체크
limit = 0
for row in df.total_tokens:
    if limit > 10:
        break
    for tup in row:
        if tup[1]=='VerbPrefix':
            print(tup)
            limit += 1

('잘', 'VerbPrefix')
('안', 'VerbPrefix')
('안', 'VerbPrefix')
('못', 'VerbPrefix')
('잘', 'VerbPrefix')
('안', 'VerbPrefix')
('잘', 'VerbPrefix')
('안', 'VerbPrefix')
('안', 'VerbPrefix')
('안', 'VerbPrefix')
('안', 'VerbPrefix')
('잘', 'VerbPrefix')
('안', 'VerbPrefix')


### 익게2 데이터로부터 BoW 생성

In [585]:
#Total Token's Bag of Words
corpus = []
for column in ['total_tokens']:
    for text in df.loc[df['board']=='익게2',column]:
        for word in text:
            if word[1]=='Noun':
                if word[0] not in stopwords:
                    corpus.append(word[0])

In [593]:
idx_total = {}
bow_total = []

for token in corpus:
    if token not in idx_total.keys():
        idx_total[token] = len(idx_total)
        bow_total.insert(len(idx_total)-1,1)
    else:
        bow_total[idx_total[token]] += 1
        
idx_key = list(idx_total.keys())

In [10]:
with open('./data/idx_total.pkl','rb') as f:
    idx_total = pickle.load(f)
with open('./data/bow_total.pkl','rb') as f:
    bow_total = pickle.load(f)
idx_key = list(idx_total.keys())

### 익게2의 각 날짜별 BoW 생성

In [596]:
from datetime import datetime, timedelta

#corpus for each day
cfd = pd.DataFrame(columns=['upload_date','BoW'])

for date in date_range(datetime(2021,1,1), datetime(2021,7,1)):
    bow = np.zeros(len(idx_total.keys()))
    
    for token in df.loc[(df.board=='익게2') & (df.upload_date==date),'total_tokens']:
        for word in token:
            if (word[1]=='Noun') and (word[0] not in stopwords):
                try:
                    bow[idx_total[word[0]]] += 1
                except:
                    print('Error occured')
                    
    cfd.loc[len(cfd)] = [date, bow]

In [11]:
with open('./data_l/cfd.pkl','rb') as f:
    cfd = pickle.load(f)

In [12]:
#BoW 매트릭스
BoW_mat = np.array([list(value) for value in cfd.BoW.values])
BoW_mat

array([[11.,  1., 29., ...,  0.,  0.,  0.],
       [ 0.,  0., 41., ...,  0.,  0.,  0.],
       [ 2.,  3., 42., ...,  0.,  0.,  0.],
       ...,
       [ 3.,  1., 69., ...,  0.,  0.,  0.],
       [ 1.,  3., 53., ...,  0.,  0.,  0.],
       [ 1.,  3., 54., ...,  1.,  2.,  1.]])

### 날짜별 인기 키워드 탐색

#### 지수 정의

급상승 지수
1. 전일 언급 횟수 a
2. 당일 언급된 횟수 b
3. 전체 기간 중 언급 횟수 c
4. 가중치 w0, w1, w2
5. 적당한 상수 K, 아주 작은 상수 E

w0 x log(1+(b - a)/(a + K1)) + w1 x tanh( 0.1 x (b - K2) ) - w2 x c^E

In [16]:
def relu(x):
    if x<0:
        return 0
    else:
        return x

# [w0, w1, w2, K1, K2, E]    
W = [1,1,-0.0001,10,10,1.1]

def _hot_point(a,b,c):
    x0 = np.log(relu(((b-a)/(a+W[3])))+1)
    x1 = np.tanh(0.1*(b-W[4]))
    x2 = pow(c,W[5])
    
    return W[0]*x0, W[1]*x1, W[2]*x2
    
def hot_point(a,b,c):
    x,y,z = _hot_point(a,b,c)
    return x + y + z

def pp2(a,b,idx):
    c = BoW_mat[:,idx].sum()
    x, y, z = _hot_point(a,b,c)
    print(f'상승률지수 : {x}')
    print(f'언급지수   : {y}')
    print(f'패널티     : {z}')
    print(f'최종값     : {x+y+z}')

#### 날짜/단어별 지수 columns 생성

In [266]:
#각 날짜/단어별 급상승 지수 계산
cfd['points'] = [np.zeros(len(bow_total)) for i in range(len(cfd))]

for i in range(1,len(BoW_mat)):
    for j in range(len(bow_total)):
        cfd.points[i][j] = hot_point(BoW_mat[i-1,j],BoW_mat[i,j],BoW_mat[:,j].sum())

In [267]:
#Points 매트릭스
Points = np.array([list(value) for value in cfd.points.values])
Points

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.78060801, -0.77416411, -0.49338631, ..., -0.76169416,
        -0.76180851, -0.76169416],
       [-0.50072907, -0.35457346, -0.74149917, ..., -0.76169416,
        -0.76180851, -0.76169416],
       ...,
       [-0.36101737, -0.63355764, -0.33941053, ..., -0.76169416,
        -0.76180851, -0.76169416],
       [-0.73531173, -0.44988364, -0.7579678 , ..., -0.76169416,
        -0.76180851, -0.76169416],
       [-0.73531173, -0.61693773, -0.74215272, ..., -0.62108769,
        -0.48192957, -0.62108769]])

- __불용어 추가__

In [268]:
X_list = ['데','뭐','게','익','층','실','곤','급','관','쪽','번','중','감','명','점','곳']
for idx in [idx_total[key] for key in X_list]:
    Points[:,idx] = 0

#### 일별 상위 10, 20개 키워드 추려내기

In [269]:
#index 찾아 True로 저장할 ndarray 정의
top10s = np.zeros(Points.shape, dtype=bool)
top20s = np.zeros(Points.shape, dtype=bool)

for i in range(1,len(Points)):
    #날짜별 점수 상위 10개만 추림
    limit_10 = np.sort(Points[i,:])[-10]
    limit_20 = np.sort(Points[i,:])[-20]
    for j in range(Points.shape[1]):
        if Points[i,j]>=limit_20:
            #상위 20개에 대해 True값으로 저장
            top20s[i,j] = True
            
            if Points[i,j]>=limit_10:
                #상위 10개에 대해 True값으로 저장
                top10s[i,j] = True

top10s

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [270]:
#일별 상위 10개 키워드 저장
top10_keywords = [[idx_key[int(j)] for j in np.arange(top10s.shape[1])[top10s[i]]] for i in range(len(top10s))]
#일별 상위 20개 키워드 저장
top20_keywords = [[idx_key[int(j)] for j in np.arange(top20s.shape[1])[top20s[i]]] for i in range(len(top20s))]

In [271]:
top20_keywords[1]

['선택',
 '얘',
 '배려',
 '경험',
 '룸메',
 '족보',
 '생리',
 '대학원',
 '서로',
 '헬스장',
 '밑',
 '약',
 '조언',
 '중대',
 '연구',
 '김치',
 '냉동',
 '감염',
 '모닝콜',
 '쓰레기']

#### 상위 10개 키워드 중 빈출 키워드 찾기

__top10s__의 __BoW__ 생성

In [272]:
top10s_idx = {}
top10s_bow = []

for day_keywords in top10_keywords:
    for keyword in day_keywords:
        if keyword not in top10s_idx.keys():
            top10s_idx[keyword]=len(top10s_idx)
            top10s_bow.insert(len(top10s_idx)-1,1)
        else :
            top10s_bow[top10s_idx[keyword]]+=1
            
top10s_bow = np.array(top10s_bow)

__BoW__를 __DataFrame__화

In [273]:
top10s_keys = [key for key, value in top10s_idx.items()]

df_10bow = pd.DataFrame(np.array([top10s_keys,top10s_bow]).T,
                      columns = ['keyword','count'])
df_10bow['count'] = df_10bow['count'].astype(int)

df_10bow.head()

,keyword,count
0,배려,1
1,룸메,1
2,생리,3
3,헬스장,1
4,조언,1


In [274]:
df_10bow.sort_values('count',ascending=False)[:50]

,keyword,count
114,연애,8
420,외모,8
17,변호사,8
69,문과,8
577,아빠,7
33,일본,7
223,의사,7
65,의대,6
294,대출,6
64,중국,6


#### 상위 20개 키워드 중 빈출 키워드 찾기

__top20s__의 __BoW__ 생성

In [275]:
top20s_idx = {}
top20s_bow = []

for day_keywords in top20_keywords:
    for keyword in day_keywords:
        if keyword not in top20s_idx.keys():
            top20s_idx[keyword]=len(top20s_idx)
            top20s_bow.insert(len(top20s_idx)-1,1)
        else :
            top20s_bow[top20s_idx[keyword]]+=1
            
top20s_bow = np.array(top20s_bow)

__BoW__를 __DataFrame__화

In [276]:
top20s_keys = [key for key, value in top20s_idx.items()]

df_20bow = pd.DataFrame(np.array([top20s_keys,top20s_bow]).T,
                      columns = ['keyword','count'])
df_20bow['count'] = df_20bow['count'].astype(int)

df_20bow.head()

,keyword,count
0,선택,3
1,얘,1
2,배려,2
3,경험,1
4,룸메,1


In [277]:
df_20bow.sort_values('count',ascending=False)[:50]

,keyword,count
35,변호사,14
178,결혼,13
137,문과,12
40,휴학,12
271,아빠,12
70,외모,12
24,동생,12
220,연애,11
63,미국,11
518,팬,10


__가중치 조정 기록 및 결과 저장__

In [278]:
'''
import pickle
f_idx = '2nd'
with open(f'{f_idx}_weights.pkl','wb') as f:
    pickle.dump(W,f)
with open(f'{f_idx}_cfd.pkl','wb') as f:
    pickle.dump(cfd,f)
with open(f'{f_idx}_10bow.pkl','wb') as f:
    pickle.dump(df_10bow,f)
with open(f'{f_idx}_20bow.pkl','wb') as f:
    pickle.dump(df_20bow,f)
'''

__불러오기__

In [14]:
with open('./data/10bow.pkl','rb') as f:
    df_10bow = pickle.load(f)
with open('./data/20bow.pkl','rb') as f:
    df_20bow = pickle.load(f)

---

## 월별 인기 키워드 찾기

### 작년 12월 데이터를 Load

In [80]:
tokens_20 = pd.read_csv('./data_l/ssodam2020_all_tokened_final.csv', encoding='utf-8')
#날짜 데이터를 datetime으로.
tokens_20.upload_date = pd.to_datetime(tokens_20.upload_date)
tokens_20.head()

,num,board,title,text,writer,upload_date,upload_time,view,likes,dislikes,comments,comments_writer,comments_cnt,popularity,text_tokenized,title_tokenized,comment_tokenized
0,813032,익게2,2020년 파이팅!!,모두들 파이팅,익명,2020-01-01,00:00,1375,168,0,"['올 첫글 ㅋㅋ', '아 너 뭔데 나보다 일찍 썼냐ㅡㅡ추천이나 먹어라', '첫글 ...",NaN,14,1,"[('파이팅', 'Noun')]","[('2020년', 'Number'), ('파이팅', 'Noun'), ('!!', ...","[(""['"", 'Punctuation'), ('올', 'Verb'), ('첫', '..."
1,813033,익게2,새해,ㅊㅊ,익명,2020-01-01,00:00,97,2,0,NaN,NaN,0,0,"[('ㅊㅊ', 'KoreanParticle')]","[('새해', 'Noun')]",NaN
2,813034,익게2,2020 첫 글 ㄱㅈㅇ,ㅇㅇ,익명,2020-01-01,00:00,128,1,0,NaN,NaN,0,0,"[('ㅇㅇ', 'KoreanParticle')]","[('2020', 'Number'), ('첫', 'Noun'), ('글', 'Nou...",NaN
3,813037,익게2,ㅅㅅ,ㅅㅅ,익명,2020-01-01,00:00,70,0,0,NaN,NaN,0,0,"[('ㅅㅅ', 'KoreanParticle')]","[('ㅅㅅ', 'KoreanParticle')]",NaN
4,813038,익게2,스물넷이다,뭐했다고 벌써,익명,2020-01-01,00:00,124,3,0,"['어? 너두?', '난방금전까지 스물넷..']",NaN,2,0,"[('뭐', 'Noun'), ('했다고', 'Verb'), ('벌써', 'Noun')]","[('스물넷', 'Noun'), ('이다', 'Josa')]","[(""['"", 'Punctuation'), ('?', 'Punctuation'), ..."


In [81]:
def get_noun_list_from_str(msg):
    try :
        return [tuple([re.sub("'",'',y) for y in re.findall('\'.*?\'',x)]) for x in re.findall('\(.*?\)',msg)]
    except :
        return np.NaN

In [82]:
#12월 데이터만 추려내어 처리
tokens_20_12 = tokens_20[tokens_20.upload_date.apply(lambda x: True if x.month==12 else False)]
tokens_20_12 = tokens_20_12.fillna('')
tokens_20_12.text_tokenized = tokens_20_12.text_tokenized.apply(lambda x: get_noun_list_from_str(x))
tokens_20_12.title_tokenized = tokens_20_12.title_tokenized.apply(lambda x: get_noun_list_from_str(x))
tokens_20_12.comment_tokenized = tokens_20_12.comment_tokenized.apply(lambda x: get_noun_list_from_str(x))
tokens_20_12['comments_writer_tokenized'] = [[] for _ in range(len(tokens_20_12))]
tokens_20_12.rename(columns={'comment_tokenized':'comments_tokenized'},inplace=True)
tokens_20_12.head()

,num,board,title,text,writer,upload_date,upload_time,view,likes,dislikes,comments,comments_writer,comments_cnt,popularity,text_tokenized,title_tokenized,comments_tokenized,comments_writer_tokenized
161968,1058725,익게2,엽기붕어빵ㅋㅋ,붕어빵10개 1만 6천원ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ...,익명,2020-12-01,00:06,124,0,0,['ㅈ같은 프리미엄화 좀 하지마라ㅠ'],,1,0,"[(붕어빵, Noun), (10, Number), (개, Noun), (1만, Nu...","[(엽기, Noun), (붕어빵, Noun), (ㅋㅋ, KoreanParticle)]","[("", ,), (ㅈ, KoreanParticle), (같은, Adjective),...",[]
161969,1058727,익게2,다미들 푸우욱 잘 자라옹,졸리다냥,익명,2020-12-01,00:07,143,7,0,,,0,0,"[(졸리다냥, Verb)]","[(푸우, Noun), (욱, Noun), (자라, Noun), (옹, Noun)]",[],[]
161970,1058728,익게2,나 시 써봤는데 어때,거울 속에 한 사내가 서있다사내는 거울 속 자신의 모습이 볼품없다사내는 그런 자신의...,익명,2020-12-01,00:08,245,7,0,['거사사 너때 때멈 살그... 라고요???? 말이 너무 심하시네요!! 비추드립니다...,"['뭐라는거야ㅋㅋㅋㅋ', '저도 쓰고보니 이상 거울이랑 윤동주 자화상이랑 살짝 섞인...",8,0,"[(거울, Noun), (속, Noun), (사내, Noun), (서있다, Verb...","[(시, Noun), (써, Verb), (봤는데, Verb)]","[("", ,), (사사, Noun), (때멈, Noun), (살그, Noun), (...",[]
161971,1058729,익게2,민법 한번 공부해 보고싶은데.. 인강 뭐 들어야할지 모르겠어ㅜ,세무사 민법이랑 법무사 민법...,익명,2020-12-01,00:09,221,0,2,"['로스쿨 민법 어때? - 로스쿨다미', '자기계발용으로는 적합한지 모르겠지만ㅋㅋㅋ...","['헤헤 그거 들어볼까?? 혹시 추천 강사님 물어봐도돼?ㅎ', 'ㅋㅋㅋㅋ']",8,0,"[(세무사, Noun), (민법, Noun), (법무사, Noun), (민법, No...","[(민법, Noun), (한번, Noun), (공부, Noun), (해, Verb)...","[("", ,), (로스쿨, Noun), (민법, Noun), (?, Punctuat...",[]
161972,1058730,익게2,가정내에서도 결국 사회생활하듯이 살아야할까,나는 엄마랑 감정적으로 친해지고 싶지 않아말하는 관점이 너어어어무 다르고 남 험담을...,익명,2020-12-01,00:10,376,11,0,"['솔직히 내가 그 상황이 아니니까 답은 알 수 없지만, 나라면 안 드려 이번에 드...",['진짜 고민이야 ㅠㅠ 정말 오늘 아침도 또 아빠욕하길래 못들어주겟다고 하고 나왔는...,11,0,"[(엄마, Noun), (랑, Josa), (감정, Noun), (적, Suffix...","[(가정, Noun), (내, Noun), (에서도, Josa), (사회생활, No...","[("", ,), (솔직히, Adjective), (내, Noun), (상황, Nou...",[]


In [83]:
tokens_20_12.columns

Index(['num', 'board', 'title', 'text', 'writer', 'upload_date', 'upload_time',
       'view', 'likes', 'dislikes', 'comments', 'comments_writer',
       'comments_cnt', 'popularity', 'text_tokenized', 'title_tokenized',
       'comments_tokenized', 'comments_writer_tokenized'],
      dtype='object')

In [84]:
df.columns

Index(['num', 'board', 'title', 'text', 'writer', 'upload_date', 'upload_time',
       'view', 'likes', 'dislikes', 'comments', 'comments_writer',
       'comments_cnt', 'text_tokenized', 'title_tokenized',
       'comments_tokenized', 'comments_writer_tokenized'],
      dtype='object')

In [85]:
df2 = pd.concat([tokens_20_12,df])

### 익게2 12월~6월간 BoW 생성

전체 데이터프레임을 월별로 groupby.

In [75]:
cfm_dict = dict.fromkeys(['upload_month','title_tokenized','text_tokenized','comments_tokenized','comments_writer_tokenized'])

for key in cfm_dict.keys():
    if key == 'upload_month':
        cfm_dict[key] = [12]+[i+1 for i in range(6)]
    else:
        cfm_dict[key] = [[] for i in range(7)]

cfm_dict

{'upload_month': [12, 1, 2, 3, 4, 5, 6],
 'title_tokenized': [[], [], [], [], [], [], []],
 'text_tokenized': [[], [], [], [], [], [], []],
 'comments_tokenized': [[], [], [], [], [], [], []],
 'comments_writer_tokenized': [[], [], [], [], [], [], []]}

In [658]:
_df = df2.loc[df2.board=='익게2']

for idx in range(len(_df)):
    if (idx+1)%10000==0:
        print('index :',idx,'processing...')
    row = _df.iloc[idx]
    #데이터프레임으로부터 월 인덱스 탐색
    month = 0 if row.upload_date.month==12 else row.upload_date.month
    #칼럼별 해당 인덱스에 해당되는 토큰 저장
    #50단어가 넘는 장문의 글은 제거하고 댓글과 제목만 남김.
    if len(row.text_tokenized) > 50:
        pass
    else :
        cfm_dict['text_tokenized'][month] += row.text_tokenized
    cfm_dict['title_tokenized'][month] += row.title_tokenized
    cfm_dict['comments_tokenized'][month] += row.comments_tokenized
    cfm_dict['comments_writer_tokenized'][month] += row.comments_writer_tokenized

index : 9999 processing...
index : 19999 processing...
index : 29999 processing...
index : 39999 processing...
index : 49999 processing...
index : 59999 processing...
index : 69999 processing...


In [659]:
cfm = pd.DataFrame(cfm_dict)
cfm['total_tokens'] = cfm.title_tokenized + cfm.text_tokenized + cfm.comments_tokenized + cfm.comments_writer_tokenized
cfm

,upload_month,title_tokenized,text_tokenized,comments_tokenized,comments_writer_tokenized,total_tokens
0,12,"[(엽기, Noun), (붕어빵, Noun), (ㅋㅋ, KoreanParticle)...","[(붕어빵, Noun), (10, Number), (개, Noun), (1만, Nu...","[("", ,), (ㅈ, KoreanParticle), (같은, Adjective),...",[],"[(엽기, Noun), (붕어빵, Noun), (ㅋㅋ, KoreanParticle)..."
1,1,"[(해피뉴이어, Noun), (2021년, Number), (새해, Noun), (...","[(2021년, Number), (은, Foreign), (모두, Noun), (행...","[(앗, Noun), (,, Punctuation), (성지, Noun), (다, ...","[(2, Number), (빠다, Noun), (ㅎㅎ, KoreanParticle)...","[(해피뉴이어, Noun), (2021년, Number), (새해, Noun), (..."
2,2,"[(않이, Verb), (모해, Noun), (따고, Verb), (벌써, Noun...","[(2월, Number), (이냐, Foreign), (지인, Noun), (이, ...","[(난, Noun), (마카롱, Noun), (세트, Noun), (했아, Verb...","[(오, Noun), (좋다, Adjective), (고마워, Adjective),...","[(않이, Verb), (모해, Noun), (따고, Verb), (벌써, Noun..."
3,3,"[(약, Noun), (19, Number), (저가, Noun), (코스프레, N...","[(취미, Noun), (로, Josa), (배워, Verb), (보고싶어, Ver...","[(진짜, Noun), (이형, Noun), (꾸준하네, Adjective), (,...","[(고마워, Adjective), (덕분, Noun), (에, Josa), (등업,...","[(약, Noun), (19, Number), (저가, Noun), (코스프레, N..."
4,4,"[(만우절, Noun), (이, Josa), (벌써, Noun), (끝나, Verb...","[(아쉽다, Adjective), (아쉬워, Adjective), (연애, Noun...","[(스타트, Noun), (가, Josa), (좋구만, Adjective), (ㄱㄱ...","[(나, Noun), (에게, Josa), (있다고, Adjective), (,, ...","[(만우절, Noun), (이, Josa), (벌써, Noun), (끝나, Verb..."
5,5,"[(2021년, Number), (5월, Number), (첫, Noun), (글,...","[(새로운, Adjective), (달도, Noun), (희망, Noun), (차게...","[(뭔, Modifier), (데, Noun), (벌써, Noun), (5월, Nu...","[(아, Exclamation), (진짜, Noun), (로, Josa), (그럼,...","[(2021년, Number), (5월, Number), (첫, Noun), (글,..."
6,6,"[(지금, Noun), (퇴근, Noun), (하는, Verb), (길, Noun)...","[(입사, Noun), (1년, Number), (아직, Adverb), (안된, ...","[(화이팅, Noun), (시간, Noun), (이, Josa), (지나면, Ver...","[(전망, Noun), (좋고, Adjective), (과제, Noun), (하기,...","[(지금, Noun), (퇴근, Noun), (하는, Verb), (길, Noun)..."


In [660]:
#Total Token's Bag of Words
corpus_extended = []
for column in ['total_tokens']:
    for text in cfm[column]:
        for word in text:
            try:
                if word[1]=='Noun':
                    if word[0] not in stopwords:
                        corpus_extended.append(word[0])
            except:
                pass

In [661]:
idx_extended = {}
bow_extended = []

for token in corpus_extended:
    if token not in idx_extended.keys():
        idx_extended[token] = len(idx_extended)
        bow_extended.insert(len(idx_extended)-1,1)
    else:
        bow_extended[idx_extended[token]] += 1
        
idx_extended_key = list(idx_extended.keys())

In [14]:
'''
with open('./data/idx_extended.pkl','wb') as f:
    pickle.dump(idx_extended,f)
with open('./data/bow_extended.pkl','wb') as f:
    pickle.dump(bow_extended,f)
'''
with open('./data/idx_extended.pkl','rb') as f:
    idx_extended = pickle.load(f)
with open('./data/bow_extended.pkl','rb') as f:
    bow_extended = pickle.load(f)
idx_extended_key = list(idx_extended.keys())

###  Word별 게시글 수 탐색

In [983]:
def return_words(x):
    output = []
    for token in x:
        if len(token)>1:
            output.append(token[0])
        else:
            pass
    return output

df2['total_tokens'] = df2.title_tokenized + df2.text_tokenized + df2.comments_tokenized + df2.comments_writer_tokenized
df2['_total_tokens'] = df2['total_tokens'].apply(lambda x : return_words(x))

In [1554]:
from datetime import datetime
month_filter = []
for i in range(7):
    if i==0:
        month_filter.append((datetime(2020,12,1)<=df2.upload_date) & (df2.upload_date<datetime(2021,1,1)) & (df2.board=='익게2'))
    else:
        month_filter.append((datetime(2021,i,1)<=df2.upload_date) & (df2.upload_date<datetime(2021,i+1,1)) & (df2.board=='익게2'))

import time

posts_extended = np.zeros((7,len(idx_extended)))

start_time = time.time()

for i, word in enumerate(idx_extended):
    if i%1000 == 0:
        print(i,'번째 단어 처리중...',i-1000,'번째로부터',time.time()-start_time,'초 경과.')
        start_time = time.time()
    _all_posts = df2._total_tokens.apply(lambda x: True if word in x else False)
    for j in range(0,7):
        posts_extended[j,i] = _all_posts[month_filter[j]].sum()

In [8]:
with open('./data/posts_extended.pkl','rb') as f:
    posts_extended = pickle.load(f)

### 익게2 데이터로부터 월별 BoW 생성

__BoW 생성__

In [665]:
bows = []

for month in range(7):
    bow = np.zeros(len(idx_extended.keys()))
    
    for word in cfm.iloc[month].total_tokens:
        #()등 빈 단어들은 패스
        if len(word)!=2:
            continue
        if (word[1]=='Noun') and (word[0] not in stopwords):
            try:
                bow[idx_extended[word[0]]] += 1
            except:
                print('Error occured')
                
    bows.append(bow)

cfm['BoW'] = bows

In [17]:
'''
with open('./data_l/cfm.pkl','wb') as f:
    pickle.dump(cfm,f)
'''
with open('./data_l/cfm.pkl','rb') as f:
    cfm = pickle.load(f)

### 급상승 지수 계산

idx_total & bow_total로부터 각 월마다 키워드별 급상승지수 계산

월별 급상승 지수
1. 전월 언급 횟수 a
2. 당월 언급된 횟수 b
3. 당월 게시글 작성 횟수 c
4. 전체 기간(2020.12~2021.6) 중 언급 횟수 d
5. 가중치 w0, w1, w2
6. 적당한 상수 K, 아주 작은 상수 E

w0 x log(1+(b - a)/(a + K1)) + w1 x tanh( 0.1 x (b - K2) ) - w2 x c^E

- 월별 등장 횟수는 일별 등장 횟수에 비해 양에 대한 기준치를 높여야 하므로, K1과 K2를 상향 조정

In [1021]:
posts_extended

array([[ 6., 16.,  1., ...,  0.,  0.,  0.],
       [ 4.,  6.,  1., ...,  0.,  0.,  0.],
       [ 2.,  2.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 1.,  5.,  3., ...,  0.,  0.,  0.],
       [ 9.,  6.,  1., ...,  0.,  0.,  0.],
       [ 3.,  6.,  1., ...,  1.,  1.,  1.]])

In [1027]:
BoW_mat

array([[ 36., 194.,   6., ...,   0.,   0.,   0.],
       [ 18.,  29.,   4., ...,   0.,   0.,   0.],
       [  4.,  20.,   0., ...,   0.,   0.,   0.],
       ...,
       [  3.,  38.,   8., ...,   0.,   0.,   0.],
       [ 22.,  18.,   3., ...,   0.,   0.,   0.],
       [ 12.,  39.,   3., ...,   3.,   3.,   3.]])

In [365]:
def relu(x):
    if x<0:
        return 0
    else:
        return x
    
def sigmoid2(x):
    return 1/( 1+np.exp( -(x-5) ) )

#   [w0,   w1,  w2,  w3,    w4,  K1,  K2, K3, E]    
W = [1.5, 0.3, 1.3, 0.5, -0.05, 10, 100, 10, 1.002]

def _hot_point3(a,b,c,d,e):
    x0 = sigmoid2((b-a)/(a+W[5]))
    x1 = np.tanh(0.05*(b-W[6]))-0.5
    x2 = sigmoid2((d-c)/(c+W[7]))
    x3 = np.tanh(0.1*(d-20))-0.7
    x4 = pow(W[8],e) - 1
    
    return W[0]*x0, W[1]*x1, W[2]*x2, W[3]*x3, W[4]*x4
    
def hot_point3(a,b,c,d,e):
    x,y,z,w,v = _hot_point3(a,b,c,d,e)
    return x + y + z + w + v

def pp3(a,b,c,d,idx):
    e = BoW_mat[:,idx].sum()
    x, y, z, w, v = _hot_point3(a,b,c,d,e)
    print(f'상승률지수 : {x}')
    print(f'언급지수   : {y}')
    print(f'게시글상승 : {z}')
    print(f'게시글수   : {w}')
    print(f'패널티     : {v}')
    print(f'최종값     : {x+y+z+w+v}')

In [366]:
#BoW 매트릭스
BoW_mat = np.array([list(value) for value in cfm.BoW.values])
BoW_mat

array([[ 36., 194.,   6., ...,   0.,   0.,   0.],
       [ 18.,  29.,   4., ...,   0.,   0.,   0.],
       [  4.,  20.,   0., ...,   0.,   0.,   0.],
       ...,
       [  3.,  38.,   8., ...,   0.,   0.,   0.],
       [ 22.,  18.,   3., ...,   0.,   0.,   0.],
       [ 12.,  39.,   3., ...,   3.,   3.,   3.]])

In [367]:
#각 날짜/단어별 급상승 지수 계산
cfm['points'] = [np.zeros(len(bow_extended)) for i in range(len(cfm))]

for i in range(1,len(BoW_mat)):
    for j in range(len(bow_extended)):
        cfm.points[i][j] = hot_point3(BoW_mat[i-1,j],BoW_mat[i,j],
                                      posts_extended[i-1,j],posts_extended[i,j],
                                      BoW_mat[:,j].sum())

In [368]:
#Points 매트릭스
Points = np.array([list(value) for value in cfm.points.values])
Points

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-1.26481776, -1.2968079 , -1.26402542, ..., -1.26354717,
        -1.26354717, -1.26354717],
       [-1.27580816, -1.31301699, -1.26999508, ..., -1.26354717,
        -1.26354717, -1.26354717],
       ...,
       [-1.27342948, -1.28727906, -1.25261751, ..., -1.26354717,
        -1.26354717, -1.26354717],
       [-1.16918425, -1.29200404, -1.26651268, ..., -1.26354717,
        -1.26354717, -1.26354717],
       [-1.27101071, -1.26587024, -1.26285661, ..., -1.25525361,
        -1.25525361, -1.25525361]])

__추가 불용어 정의__

In [369]:
#도배된 단어 등 제외
for x_word in ['는','걍','을해','얀','인데','꿀꿀','끝내','쿠','원','쿠나','게로','사지','능','조만간','알폰소','아론']:
    Points[:,idx_extended[x_word]] = 0.

### 상위 키워드 추리기

In [370]:
#index 찾아 True로 저장할 ndarray 정의
top10s = np.zeros(Points.shape, dtype=bool)
top20s = np.zeros(Points.shape, dtype=bool)

for i in range(1,len(Points)):
    #날짜별 점수 상위 10개만 추림
    limit_10 = np.sort(Points[i,:])[-10]
    limit_20 = np.sort(Points[i,:])[-20]
    for j in range(Points.shape[1]):
        if Points[i,j]>=limit_20:
            #상위 20개에 대해 True값으로 저장
            top20s[i,j] = True
            
            if Points[i,j]>=limit_10:
                #상위 10개에 대해 True값으로 저장
                top10s[i,j] = True

top10s

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [371]:
#일별 상위 10개 키워드 저장
top10_keywords = [[idx_extended_key[int(j)] for j in np.arange(top10s.shape[1])[top10s[i]]] for i in range(len(top10s))]
#일별 상위 20개 키워드 저장
top20_keywords = [[idx_extended_key[int(j)] for j in np.arange(top20s.shape[1])[top20s[i]]] for i in range(len(top20s))]

In [372]:
top10_keywords

[[],
 ['눈사람', '능검', '공론', '재래시장', '페스', '페이크', '정인', '로토', '팬픽', '강민철'],
 ['증원', '쿼터', '뻔뻔', '연강', '괴롭힘', '뻔선', '뻔후', '폭로', '학폭', '학교폭력'],
 ['피셋', '결석', '벚꽃', '로나', '검찰', '토지', '신도시', '투기', '단태', '브레이브걸스'],
 ['파상', '벼락치기', '클로즈', '분포', '슈퍼리그', '비제', '권혁', '만우절', '서예지', '김정현'],
 ['참가자', '퇴소', '자진', '파이', '정민', '퍼즐', '이루리', '공혁준', '또즐', '니갸르'],
 ['부정행위', '텀페', '중회', '독일', '관회', '메리트', '광학', '팬덤', '유로', '소희']]

In [373]:
top20_keywords

[[],
 ['눈사람',
  '능검',
  '공론',
  '재래시장',
  '페스',
  '페이크',
  '손예진',
  '삽자루',
  '정인',
  '로토',
  '팬픽',
  '보따리',
  '강민철',
  '불평',
  '와이번스',
  '동포',
  '담배값',
  '냉면',
  '아나콘다',
  '오줌싸개'],
 ['증원',
  '물성',
  '성폭행',
  '쿼터',
  '설날',
  '뻔뻔',
  '사인',
  '연강',
  '괴롭힘',
  '뻔선',
  '연휴',
  '방관',
  '뻔후',
  '폭로',
  '학폭',
  '학교폭력',
  '탈세',
  '동물원',
  '김연경',
  '역군'],
 ['장관',
  '피셋',
  '에이프릴',
  '결석',
  '벚꽃',
  '로나',
  '검찰',
  '역주행',
  '토지',
  '신도시',
  '서진',
  '투기',
  '유신',
  '롤린',
  '단태',
  '브레이브걸스',
  '마사',
  '몰수',
  '하윤',
  '꼬북좌'],
 ['부정행위',
  '파상',
  '벼락치기',
  '테뱅',
  '클로즈',
  '토트넘',
  '분포',
  '슈퍼리그',
  '비제',
  '너비',
  '간담',
  '애정',
  '작',
  '윽박',
  '결식',
  '권혁',
  '만우절',
  '서예지',
  '진순',
  '김정현'],
 ['코난',
  '범인',
  '전기',
  '참가자',
  '학생회',
  '성당',
  '퇴소',
  '사물함',
  '실종',
  '자진',
  '파이',
  '어린이',
  '에어컨',
  '정민',
  '퍼즐',
  '이루리',
  '캣맘',
  '공혁준',
  '또즐',
  '니갸르'],
 ['부정행위',
  '경통',
  '텀페',
  '중회',
  '떡밥',
  '국무',
  '독일',
  '할미',
  '사프컴',
  '관회',
  '반타작',
  '메리트',
  '광학',
  '팬덤',
  '잉글랜드',
  '현주',
  '유로',

__월별 키워드 보기__

In [399]:
month = 1

for month in range(1,7):
    display([(idx_extended_key[arg],Points[month,arg]) for arg in np.flip(np.argsort(Points[month])) if Points[month,arg]>0.4])
    print()

[('능검', 2.260767814533165),
 ('눈사람', 1.6846429495194757),
 ('페스', 1.610230077040688),
 ('로토', 1.0773097870681805),
 ('팬픽', 1.0042855362774332),
 ('페이크', 1.0042390116242759),
 ('강민철', 0.8337913365931837),
 ('재래시장', 0.8249923193467364),
 ('정인', 0.7824369188747704),
 ('공론', 0.7237889856652208),
 ('냉면', 0.7000926140740011),
 ('오줌싸개', 0.6285326091369418),
 ('아나콘다', 0.5467134683125027),
 ('와이번스', 0.5331136670134958),
 ('보따리', 0.5063553456897618),
 ('삽자루', 0.4552971322576629)]

[('뻔뻔', 2.9060766694286704),
 ('학폭', 2.17729433603298),
 ('뻔선', 1.8340011697949712),
 ('학교폭력', 1.7158133778593483),
 ('쿼터', 1.5240046743778157),
 ('증원', 1.4487172599266391),
 ('뻔후', 1.238929017862722),
 ('폭로', 1.2205242316051437),
 ('괴롭힘', 1.1032633441401642),
 ('연강', 0.9081321681763606),
 ('동물원', 0.877751313243363),
 ('탈세', 0.8742128362653893),
 ('김연경', 0.8676058012943388),
 ('물성', 0.8389613842387756),
 ('역군', 0.8090530675135683),
 ('사인', 0.7851861517363047),
 ('연휴', 0.6764222193690317),
 ('방관', 0.6759785392670195),
 ('성폭행', 0.6582000882432335),
 ('설날', 0.6501837356569178),
 ('돌고래', 0.6475324203931211),
 ('매매혼', 0.645106973790332),
 ('강인원', 0.6242970899004467),
 ('박물관', 0.5779721602587583),
 ('뜻뜻', 0.5572805601042105),
 ('이진욱', 0.5127268180871906),
 ('읏', 0.4925825322501887),
 ('박혜수', 0.48565225162724185),
 ('명절', 0.48543285974844463),
 ('연가', 0.4368597579045375),
 ('라미', 0.42408578661221585),
 ('학보', 0.423844042948899),
 ('세뱃돈', 0.41938121923030325),
 ('분반', 0.41502840050637846)]

[('브레이브걸스', 2.22933911185193),
 ('단태', 2.0792541562070443),
 ('피셋', 1.703284982834478),
 ('신도시', 1.642462685846044),
 ('투기', 1.6165370239971752),
 ('검찰', 1.5690809043020542),
 ('결석', 1.5168880880759443),
 ('토지', 1.478933863061037),
 ('벚꽃', 1.4000063024463174),
 ('로나', 1.3672450068133981),
 ('마사', 1.3500116675752236),
 ('몰수', 1.2590418314014218),
 ('롤린', 1.2443783911042554),
 ('하윤', 1.2395557600221048),
 ('에이프릴', 1.2199456431016544),
 ('장관', 1.099773246730592),
 ('역주행', 1.0995421181074967),
 ('유신', 1.049941839395576),
 ('서진', 0.99970779535535),
 ('꼬북좌', 0.9836643072225276),
 ('수사', 0.9438734921055012),
 ('압수수색', 0.9182709420771997),
 ('자진', 0.8843710011701326),
 ('소급', 0.8766788875087869),
 ('버닝썬', 0.828980397868778),
 ('민주화', 0.8229285479641979),
 ('성매매', 0.811031207467534),
 ('룸살롱', 0.7873029929898611),
 ('향영', 0.7253117704861348),
 ('그래미', 0.7150509382790209),
 ('박수홍', 0.6941311631646634),
 ('임효준', 0.6317820125674505),
 ('수련', 0.614662265168781),
 ('쁘걸', 0.6074678333528786),
 ('동선', 

[('서예지', 1.9022807834692488),
 ('슈퍼리그', 1.6895519677147233),
 ('클로즈', 1.6880632934864688),
 ('김정현', 1.2730761500460281),
 ('비제', 1.1448339147424684),
 ('만우절', 1.1299672608995588),
 ('벼락치기', 0.8972771472427674),
 ('권혁', 0.4997246313270325),
 ('파상', 0.4670631125631868),
 ('분포', 0.4465461535817043),
 ('너비', 0.43658398039753943)]

[('참가자', 2.0620549547283336),
 ('퍼즐', 2.061422982385385),
 ('공혁준', 2.0193505845033326),
 ('이루리', 1.7752761958177912),
 ('정민', 1.758700161419743),
 ('퇴소', 1.7352013308542047),
 ('또즐', 1.7010357834601493),
 ('니갸르', 1.5548242004092578),
 ('파이', 1.5152887325168194),
 ('자진', 1.5046535945302328),
 ('사물함', 1.4891681972729147),
 ('학생회', 1.4786129546458162),
 ('전기', 1.2066400820413237),
 ('범인', 1.0835541224993812),
 ('코난', 1.0285868555594113),
 ('성당', 0.9012279189844271),
 ('에어컨', 0.8047932944798283),
 ('어린이', 0.7861394025069605),
 ('실종', 0.7526812093728193),
 ('캣맘', 0.7436837301002479),
 ('무죄', 0.6489785383501229),
 ('어버이날', 0.6076281219154295),
 ('약육강식', 0.5902651138496333),
 ('호날두', 0.574112523473142),
 ('퇴', 0.5604843885505018),
 ('육지', 0.5199889921983183),
 ('어린이날', 0.4606035918840763),
 ('최면', 0.4170355382815962)]

[('텀페', 1.606603203188053),
 ('관회', 1.3746085911616708),
 ('광학', 1.2967518608025441),
 ('독일', 1.242447823527372),
 ('팬덤', 1.201700999169101),
 ('소희', 1.1362068081919847),
 ('부정행위', 1.094260938677591),
 ('유로', 0.9619952825122491),
 ('메리트', 0.9043947269187297),
 ('중회', 0.6250872479194971),
 ('경통', 0.60320749747943),
 ('국무', 0.566041043992044),
 ('사프컴', 0.5319942501084838),
 ('현주', 0.5215723113458508),
 ('에고', 0.49777081290346725),
 ('잉글랜드', 0.49256289396082537),
 ('할미', 0.47724325105923293),
 ('반타작', 0.45792501046152534),
 ('쁘걸', 0.4462492446186387),
 ('떡밥', 0.4452337671503154),
 ('미분', 0.4445983569019486),
 ('곡선', 0.44408952348419106),
 ('리믹스', 0.4418088480118401),
 ('펩시', 0.4320645626434206),
 ('미적분', 0.4170023692209237),
 ('프제', 0.41329545502830006),
 ('열역학', 0.4113587420406447)]

### 단어별 검토

In [450]:
_word = '피셋'

- 월별 점수

In [451]:
Points[:,idx_extended[_word]]

array([ 0.        , -1.21918944, -1.37305272,  1.70328498, -1.34172567,
       -1.42083641, -1.34372986])

- 전체 등장 횟수

In [452]:
bow_extended[idx_extended[_word]]

746

- 월별 등장 횟수

In [453]:
BoW_mat[:,idx_extended[_word]]

array([ 79.,  50.,  36., 489.,  45.,  11.,  36.])

- 월별 게시글 수

In [454]:
posts_extended[:,idx_extended[_word]]

array([11., 14.,  8., 51., 10.,  4.,  8.])

- 지수 확인

In [447]:
end = 6
pp3(BoW_mat[end-1,idx_extended[_word]],BoW_mat[end,idx_extended[_word]],
    posts_extended[end-1,idx_extended[_word]], posts_extended[end,idx_extended[_word]],
    idx_extended[_word])

상승률지수 : 1.463536799078829
언급지수   : 0.04921103108035467
게시글상승 : 0.0565289497131827
게시글수   : -0.251312339887548
패널티     : -0.021212579182274108
최종값     : 1.2967518608025441


- 내용 검색 툴

In [448]:
def search_cd(word, x):
    try:
        return True if word in [y[0] for y in x] else False
    except:
        return False

def s_search(word, month, show_urls=True):
    df_month = df2[df2.upload_date.apply(lambda x: True if x.month==month else False)]
    title_cd = df2.title_tokenized.apply(lambda x: search_cd(word,x))
    text_cd = df2.text_tokenized.apply(lambda x: search_cd(word,x))
    comments_cd = df2.comments_tokenized.apply(lambda x: search_cd(word,x))
    comments_writer_cd = df2.comments_writer_tokenized.apply(lambda x: search_cd(word,x))
    df = df_month[title_cd | text_cd | comments_cd | comments_writer_cd]
    if show_urls==True:
        [print('http://www.ssodam.com/content/'+str(number)) for number in df.num]
    return df

def numofposts(word, month):
    df = s_search(word,month,show_urls=False)
    return len(df)

In [449]:
s_search('광학',6)

http://www.ssodam.com/content/1177711
http://www.ssodam.com/content/1177713
http://www.ssodam.com/content/1178229
http://www.ssodam.com/content/1183701
http://www.ssodam.com/content/1183764
http://www.ssodam.com/content/1184512
http://www.ssodam.com/content/1185045
http://www.ssodam.com/content/1185192
http://www.ssodam.com/content/1185739
http://www.ssodam.com/content/1186858
http://www.ssodam.com/content/1187343
http://www.ssodam.com/content/1187680
http://www.ssodam.com/content/1187685
http://www.ssodam.com/content/1187688
http://www.ssodam.com/content/1187702
http://www.ssodam.com/content/1188046
http://www.ssodam.com/content/1188071
http://www.ssodam.com/content/1188279
http://www.ssodam.com/content/1190762
http://www.ssodam.com/content/1190862
http://www.ssodam.com/content/1190890
http://www.ssodam.com/content/1191093
http://www.ssodam.com/content/1191672
http://www.ssodam.com/content/1197596
http://www.ssodam.com/content/1200076


,num,board,title,text,writer,upload_date,upload_time,view,likes,dislikes,comments,comments_writer,comments_cnt,popularity,text_tokenized,title_tokenized,comments_tokenized,comments_writer_tokenized
63062,1177711,익게2,현대광학 0601 녹화하신 분 있나여,오캠 녹화햇는데 오류나서 파일이 날라갓네요사례하겟읍니다,익명,2021-06-01,09:52,80.0,0.0,1.0,,,0.0,NaN,"[(오, Modifier), (캠, Noun), (녹화, Noun), (햇, Nou...","[(현대, Noun), (광학, Noun), (0601, Number), (녹화, ...",[],[]
63063,1177713,자유게시판,현대광학 0601 녹화본 구합니다,기프티콘 사례하겠습니다,사쿠라이,2021-06-01,09:55,176.0,1.0,4.0,이왜비,,2.0,NaN,"[(기프티콘, Noun), (사례, Noun), (하겠습니다, Verb)]","[(현대, Noun), (광학, Noun), (0601, Number), (녹화, ...","[(이, Determiner), (왜비, Noun)]",[]
63389,1178229,익게1,현대광학 ㅈㅈㅇ 교수님 기말 족보 구합니다,사례하겠습니다 댓글주세요,익명,2021-06-01,23:34,53.0,0.0,0.0,,,0.0,NaN,"[(사례, Noun), (하겠습니다, Verb), (댓글, Noun), (주세요, ...","[(현대, Noun), (광학, Noun), (ㅈㅈㅇ, KoreanParticle)...",[],[]
66955,1183701,익게2,물리과 핫한기념 전공 선택과목 추천좀 해주실 분,현재 전선은 광학 하나 들은 5학기생임앞으로전산물리학고체물리정도 들을거 같은데일반상...,익명,2021-06-09,23:49,271.0,0.0,0.0,"일단 전산 고체 꿀잼, 만일 반도체로 취업하고자 한다면 고체는 무적권, ㅇㅇ 파이썬...","고체 학부 치업하려면 무적건임, 전산은 머하는 가목임 혹시 몰라서 이번에 파이썬 ...",27.0,NaN,"[(현재, Noun), (전선, Noun), (은, Josa), (광학, Noun)...","[(물리, Noun), (과, Josa), (핫, Noun), (한, Determi...","[(일단, Noun), (전산, Noun), (고체, Noun), (꿀잼, Noun...","[(고체, Noun), (학부, Noun), (치, Noun), (업하려면, Ver..."
66981,1183764,익게2,물리과 살려줘,양자고 광학이고 열역학이고 하나도 모르겠어 ...,익명,2021-06-10,00:51,194.0,0.0,0.0,"광학은 진짜 그랫으면 좋겟는데 어림도업지 ㅋㅋㅋ, 양자 정성적으로 해석하는 문제 하...","ㄹㅇㅋㅋ 항상 조져지는건 나였고, 양자는 그나마 문제 경향성을 알듯말듯 하고 광학은...",8.0,NaN,"[(양자, Noun), (고, Josa), (광학, Noun), (이고, Josa)...","[(물리, Noun), (과, Josa), (살려줘, Verb)]","[(광학, Noun), (은, Josa), (진짜, Noun), (그랫으, Noun...","[(ㄹㅇㅋㅋ, KoreanParticle), (항상, Noun), (조져지는건, A..."
67433,1184512,익게2,물리학과 광학 기말 족보 있는 사람 있음,치킨 사례함,익명,2021-06-10,22:54,210.0,5.0,1.0,"광학 족보 엄청 탐, 문제는 족보 있는 애들이 많다는거임, 물리과까지 족보에 잠식당...","ㄹㅇ, 절평인데 좀 돌려보자고, ㄴ 서담 자료실에 올려줘도 치킨 선사함,",10.0,NaN,"[(치킨, Noun), (사례, Noun), (함, Noun)]","[(물리학, Noun), (과, Josa), (광학, Noun), (기, Modif...","[(광학, Noun), (족보, Noun), (엄청, Adverb), (탐, Ver...","[(ㄹㅇ, KoreanParticle), (,, Punctuation), (절평, ..."
67749,1185045,익게2,광학 다들 중간 숙제 몇점나옴,나 30나왓는에 ㅈ댄거냐,익명,2021-06-11,17:33,84.0,0.0,0.0,"35ㅋㅋ, 20점이었나ㅌㅋ, 저는 40인데 다들 비스무리 하구먼",중간 몇점이엇냐 나 딱 평균ㅋㅋㅋ,4.0,NaN,"[(나, Noun), (30, Number), (나왓는, Noun), (에, Jos...","[(광학, Noun), (다, Adverb), (들, Verb), (중간, Noun...","[(35, Number), (ㅋㅋ, KoreanParticle), (,, Punct...","[(중간, Noun), (몇, Modifier), (점, Noun), (이, Det..."
67837,1185192,익게2,광학 이거 어케푸는거임,이랑 랑 머선 차이인지 모르겠네이 문제라도 맞춰야될거같은데어케푸는지 모르겟다 ㅈ망한듯,익명,2021-06-11,20:57,164.0,0.0,0.0,"각 광학장치에 대한 조온스 매트릭스를 곱하시오, 그러면 들어오는 바앙향에 따라 이 ...",", ㅇㅎ 이해함, 기말시험에 나온다고 미리 알려주신 문제인데 못건드리겟음ㅋㅋㅋ, 족...",14.0,NaN,"[(이랑, Josa), (랑, Josa), (머선, Noun), (차이, Noun)...","[(광학, Noun), (이, Determiner), (거, Noun), (어케, ...","[(각, Noun), (광학, Noun), (장치, Noun), (에, Josa),...","[(,, Punctuation), (ㅇㅎ, KoreanParticle), (이해, ..."
68196,1185739,익게2,양자공부하다가 광학 하니까 개재밋네,이게 물리지,익명,2021-06-12,16:30,129.0,0.0,0.0,"흠 난 광학보다 양자가 훨 재밌었음, 광학은 양자보단 직관적이라 덜 짜증나는거 같음",양자그켬,3.0,NaN,"[(이, Noun), (게, Josa), (물리, Noun), (지, Josa)]","[(양자, Noun), (공부, Noun), (하, Suffix), (다가, Nou...","[(흠, Noun), (난, Noun), (광학, Noun), (보다, Josa),...","[(양자, Noun), (그, Noun), (켬, Verb)]"
68955,1186858,익게2,물리과 살려주,헥트 광학 교재 부랴부랴 풀고있는데 귀찮고 힘들어열역학은 증명문제 안 나올거 같아서...,익명,2021-06-14,04:59,199.0,2.0,0.0,"숙제 다 풀었다고 씹가능이네, 그정도면 적어도 중간 이상은 할 수 있을거야, 다 햇...",근데 풀다풀다 답지만 하루종일 붙들고 있었던 문제도 많고 정작 어떻게 푸는건지 잘 ...,6.0,NaN,"[(헥트, Noun), (광학, Noun), (교재, Noun), (부랴부랴, No...","[(물리, Noun), (과, Josa), (살려주, Verb)]","[(숙제, Noun), (다, Adverb), (풀었다고, Verb), (씹, Ve...","[(근데, Adverb), (풀다, Noun), (풀다, Noun), (답지, No..."


---

,num,board,title,text,writer,upload_date,upload_time,view,likes,dislikes,comments,comments_writer,comments_cnt,text_tokenized,title_tokenized,comments_tokenized,comments_writer_tokenized
0,1082901,익게2,해피뉴이어,2021년은 모두 행복한 한해가 되길,익명,2021-01-01,00:00,1343.0,195.0,0.0,"앗, 성지다, 첫글 ㅊㅊ, 추천 준다, 내 24살은 인생에서 가장 빛나는 날로 만들...",,20.0,"[(2021년, Number), (은, Foreign), (모두, Noun), (행...","[(해피뉴이어, Noun)]","[(앗, Noun), (,, Punctuation), (성지, Noun), (다, ...",[]
1,1082902,익게2,2021년 새해복 많이받으세요,모든 일이 잘 되기를 12시 땡,익명,2021-01-01,00:00,107.0,3.0,0.0,늦었네,"2빠다 ㅎㅎ, 내년에 도전한다",3.0,"[(모든, Noun), (일이, Modifier), (잘, Verb), (되, Ve...","[(2021년, Number), (새해, Noun), (복, Noun), (많이, ...","[(늦었네, Verb)]","[(2, Number), (빠다, Noun), (ㅎㅎ, KoreanParticle)..."
2,1082903,익게2,첫글은 내꼬,예비회계사 나다미,익명,2021-01-01,00:00,139.0,1.0,0.0,"ㄹㅇ 1등이네 ㅋㅋ, 실패",ㅜㅜ,3.0,"[(예비, Noun), (회계사, Noun), (나, Noun), (다미, Noun)]","[(첫, Modifier), (글, Noun), (은, Josa), (내꼬, Noun)]","[(ㄹㅇ, KoreanParticle), (1등, Number), (이네, Fore...","[(ㅜㅜ, KoreanParticle)]"
3,1082905,익게2,땡ㅎㅎㅎㅎㅎㅎㅎ,1등,익명,2021-01-01,00:00,39.0,0.0,0.0,,,0.0,"[(1등, Number)]","[(땡, Noun), (ㅎㅎㅎㅎㅎㅎㅎ, KoreanParticle)]",[],[]
4,1082906,익게2,어디 카운트 다운 하는 곳 없냐,언제바껴,익명,2021-01-01,00:00,84.0,0.0,0.0,232번 불교방송,,1.0,"[(언, Modifier), (제바, Noun), (껴, Verb)]","[(어디, Noun), (카운트, Noun), (다운, Noun), (하는, Ver...","[(232, Number), (번, Noun), (불교, Noun), (방송, No...",[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77975,1200285,익게2,아버지께서 칼럼 쓰셨는데 댓글 한 번씩만 달아줄 수 있을까,2222408544049우리 아버지께서 이번에 짧은 칼럼처럼 블로그에 쓰시는 거 맡...,익명,2021-06-30,23:43,5802.0,374.0,11.0,"비댓으로 달래 너무 귀엽다, 이 글 조회수 늘리려면 제목 바꾸는거 추천 광고글인줄 ...","헉 피드백 반영했어 너무 고마워, 엇 아마 확인은 못하실 텐데 비댓이든 공개댓이든 ...",75.0,"[(2222408544049, Number), (우리, Noun), (아버지, No...","[(아버지, Noun), (께서, Josa), (칼럼, Noun), (쓰셨는데, V...","[(비댓, Noun), (으로, Josa), (달래, Noun), (너무, Adve...","[(헉, Adverb), (피드백, Noun), (반영, Noun), (했어, Ve..."
77976,1200287,익게2,프로포폴 하면 기분이 좋음,하는거 신기하네하면 힙해보여서 하는건가,익명,2021-06-30,23:47,373.0,1.0,1.0,썰로 들은 거라 정확하지는 않는데 지방흡입할 때 고통 줄이려고 프로포폴 투약하다가 ...,,5.0,"[(하는거, Verb), (신기하네하면, Adjective), (힙, Noun), ...","[(프로포폴, Noun), (하면, Verb), (기분, Noun), (이, Jos...","[(썰로, Verb), (들은, Verb), (거, Noun), (라, Josa),...",[]
77977,1200288,익게2,취업 후기 쓰려는데 안올라간다ㅠ,손다쳐서 폰으로 열심히 적었는데나한테왜 이래ㅠ혹시 뭔가 지켜야 하는 양식이 있나요ㅠ...,익명,2021-06-30,23:49,387.0,3.0,0.0,"중간에 임티 넣었었어, 임티, 1 이모티콘 빼기2 사진 많으면 업로드 될 동안 조금...",이건 되네,7.0,"[(손, Noun), (다쳐서, Verb), (폰, Noun), (으로, Josa)...","[(취업, Noun), (후기, Noun), (쓰려는데, Verb), (안, Ver...","[(중간, Noun), (에, Josa), (임티, Noun), (넣었었어, Ver...","[(이건, Noun), (되네, Verb)]"
77978,1200291,익게2,간떨어지는동거 보는 사람,아 이번주 기대했는데 예고편...,익명,2021-06-30,23:55,138.0,6.0,0.0,"너무 오글거리는데 장기용때매 봄 ㅎㅎ, 난 계선우땜에 봐ㅠㅠ, 나도 계선우 때문에 ...",,3.0,"[(아, Exclamation), (이번, Noun), (주, Noun), (기대했...","[(간, Noun), (떨어지는, Verb), (동거, Noun), (보는, Ver...","[(너무, Adverb), (오글거리는데, Verb), (장기, Noun), (용때...",[]


## ETC

In [962]:
Posts_mat = np.zeros((7,len(idx_extended)))
for i, key in enumerate(idx_extended):
    print(i)
    for j in range(1,7):
        print(j)
        Posts_mat[j,i] = numofposts(key,i)

0
1
2
3
4
5
6
1
1
2
3
4
5
6
2
1
2
3
4
5
6
3
1
2
3
4
5
6
4
1


KeyboardInterrupt: 

In [ ]:
with open('posts_mat.pkl','wb') as f:
    pickle.dump(Posts_mat,f)